In [ ]:
#!/usr/bin/python

# Standard imports
import cv2
import numpy as np;

# Read image
img = cv2.imread('/home/garner1/Work/dataset/cellImages/image52/equalised_iMS266_20190426_001.sub52.tif', cv2.IMREAD_GRAYSCALE)
# img = cv2.imread('/home/garner1/Work/dataset/cellImages/image52/Images_Patient_52/equalised/equalised_iMS266_20190426_001.sub112.tif', cv2.IMREAD_GRAYSCALE)
# img = cv2.imread('/home/garner1/Work/dataset/cellImages/image52/iMS266_20190426_001.sub52.jpg', cv2.IMREAD_GRAYSCALE)

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 32
params.maxThreshold = 256
params.thresholdStep = 1
params.minDistBetweenBlobs = 0
# Filter by color.
params.filterByColor = True
params.blobColor = 255
# Filter by Area.
params.filterByArea = True
params.minArea = 16
params.maxArea = 100*params.minArea

# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.0
# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.0
params.maxConvexity = 1.0
# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.0

# Create a detector with the parameters
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3 :
    detector = cv2.SimpleBlobDetector(params)
else : 
    detector = cv2.SimpleBlobDetector_create(params)

In [ ]:
# Detect blobs.
im = img[:,:]
keypoints = detector.detect(im)

In [ ]:
# import seaborn as sns; sns.set()
# from matplotlib import pyplot as plt
# # Draw detected blobs as red circles.
# # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
# # the size of the circle corresponds to the size of blob

# im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# cv2.imshow("Keypoints", im_with_keypoints)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
import umap

X = np.asarray([keypoints[idx].pt for idx in range(len(keypoints))])
node_size = np.asarray([keypoints[idx].size for idx in range(len(keypoints))])

'''2D embedding'''
n_neighbors = 10

'''
Returns
-------
fuzzy_simplicial_set: coo_matrix
A fuzzy simplicial set represented as a sparse matrix. The (i,
j) entry of the matrix represents the membership strength of the
1-simplex between the ith and jth sample points.
'''
mat = umap.umap_.fuzzy_simplicial_set(
    X,
    n_neighbors,
    random_state=np.random.RandomState(seed=42),
    metric='euclidean',
    metric_kwds={},
    knn_indices=None,
    knn_dists=None,
    angular=False,
    set_op_mix_ratio=1.0,
    local_connectivity=1.0,
    verbose=False
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import networkx as nx
sns.set(style='white', rc={'figure.figsize':(50,50)})
G = nx.from_scipy_sparse_matrix(mat)
eset = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 0.0]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
# pos = nx.spring_layout(G)  # positions for all nodes
pos = X
# nodes
nx.draw_networkx_nodes(G, pos, node_size=node_size,alpha=1.0)
# edges
# nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, edge_color='r')
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights, edge_color='r')
# nx.draw_networkx_edges(G, pos, edgelist=esmall,
#                        width=6, alpha=0.5, edge_color='b', style='dashed')
# labels
# nx.draw_networkx_labels(G, pos, font_size=20, font_family='sans-serif')
plt.axis('off')
plt.imshow(img,cmap='gray')
plt.show()


In [ ]:
from node2vec import Node2Vec

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(G, dimensions=10, walk_length=20, num_walks=200, workers=24)  # Use temp_folder for big graphs

In [ ]:
# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

In [ ]:
embedding = np.asarray([model.wv[key] for idx, key in enumerate(model.wv.vocab)])

In [ ]:
# '''2D Visualization'''
# fig, ax = plt.subplots()

# ax.scatter(embedding[:, 0], embedding[:, 1], s=10)

# plt.show()

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_samples=None,min_cluster_size=50)
clusterer.fit(embedding)

# sns.set(style='white', rc={'figure.figsize':(10,8)})
# clustered = (clusterer.labels_ >= 0)
# plt.scatter(embedding[~clustered, 0],
#             embedding[~clustered, 1],
#             c=(0.5, 0.5, 0.5),
#             s=2,
#             alpha=0.5)
# plt.scatter(embedding[clustered, 0],
#             embedding[clustered, 1],
#             c=clusterer.labels_[clustered],
#             s=2,
#             cmap='Spectral');


In [ ]:
set(clusterer.labels_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import networkx as nx

sns.set(style='white', rc={'figure.figsize':(50,50)})

G = nx.from_scipy_sparse_matrix(mat)
eset = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] > 0.0]
weights = [d['weight'] for (u, v, d) in G.edges(data=True) if d['weight'] > 0.0]
pos = X

# nodes
colors = []
for idx in range(len(clusterer.labels_)):
    if clusterer.labels_[idx] == -1:
        colors.append('r')
    else:
        colors.append('w')
# nx.draw_networkx_nodes(G, pos, node_size=500, node_color=colors ,alpha=1.0)

nx.draw_networkx_nodes(G, pos, node_size=500, node_color=clusterer.labels_ ,alpha=1.0)

# edges
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=0.0, width=weights, edge_color='r')
plt.axis('off')
plt.imshow(img,cmap='gray')
plt.show()
